In [1]:
import xlrd 
import pubchempy as pcp
import pickle

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns

from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec, mol2sentence
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg

In [2]:
pickle_in = open("fold-0.pickle","rb")
load_dict =  pickle.load(pickle_in)

data_frame = load_dict['shuffled']

data_frame

,No.,FileHandle,SoluteName,Formula,Subset,Charge,Level1,Level2,Level3,Solvent,...,HP,OP,S,HS,OS,SP,SS,TotalArea,SoluteSMILES,SolventSMILES
1758,1759,0438pho,"diethyl2,4-dichlorophenylthiophosphate",H13C10O3P1S1CL2,[g],0,11,4,0,octanol,...,0.000,16.033,34.582,0.0,0.0,24.737,0.0,331.933,C1C2C(C(C1Cl)Cl)C3(C(=C(C2(C3(Cl)Cl)Cl)Cl)Cl)Cl,CCCCCCCCO
788,789,0071proa,propanal,H6C3O1,[g],0,2,4,1,diethylether,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,124.289,CCC=O,CCOCC
1726,1727,0210dic,"1,1-dichloro-2,2-difluoroethylmethylether",H4C3O1F2CL2,[g],0,9,1,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,178.993,CC1=CC(=O)CC(C1)(C)C,CCCCCCCCO
1939,1940,0110but,butylamine,H11C4N1,[g],0,3,1,0,secbutanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,164.243,CCCCN,CCC(C)O
1467,1468,0008noc,n-octane,H18C8,[g],0,2,1,1,methylformamide,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,243.846,CCCCCCCC,CNC=O
2110,2111,0021eth,ethene,H4C2,[a],0,2,1,4,water,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,87.417,C=C,O
739,740,0087pro,propanoicacid,H6C3O2,[g],0,2,5,0,dichloroethane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,135.630,CCC(=O)O,CC(Cl)Cl
1196,1197,0172Edi,"E-1,2-dichloroethene",H2C2CL2,[g],0,6,2,0,hexadecane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,126.855,C(=CCl)Cl,CCCCCCCCCCCCCCCC
111,112,0150mhy,m-hydroxybenzaldehyde,H6C7O2,[g],0,2,9,0,benzene,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,179.066,C1=CC(=CC(=C1)O)C=O,C1=CC=CC=C1
1820,1821,test4002,diiodomethane,H2C1I2,[g],0,15,0,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,136.531,C(I)I,CCCCCCCCO


In [3]:
# loc = ("MNSol_alldata.xlsx") 
# wb = xlrd.open_workbook(loc) 
# sheet = wb.sheet_by_index(0) 

# solutes = sheet.col_values(2)
# solvents = sheet.col_values(9)
# energies = sheet.col_values(10)

# solute_smis = []
# solvent_smis = []

# for solute in solutes[100:110]:
#     for compound in pcp.get_compounds(solute, 'name'):
#         solute_smis.append(compound.isomeric_smiles)
        
# for solvent in solvents[100:110]:
#     for compound in pcp.get_compounds(solvent, 'name'):
#         solvent_smis.append(compound.isomeric_smiles)

solute_smis = data_frame['SoluteSMILES'].tolist()
solvent_smis = data_frame['SolventSMILES'].tolist()
energies = data_frame['DeltaGsolv'].tolist()

print(len(solute_smis))
print(len(solvent_smis))
print(len(energies))

2530
2530
2530


In [4]:
solutesmis = [Chem.MolFromSmiles(x) for x in solute_smis]
solventsmis = [Chem.MolFromSmiles(x) for x in solvent_smis]

RDKit WARNING: [20:00:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:00:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:00:56] WARNING: not removing hydrogen atom without neighbors


In [5]:
solute_sentences = []
solvent_sentences = []

for solute_smi in solutesmis:
    solute_sentences.append(mol2alt_sentence(solute_smi, 1))
    
for solvent_smi in solventsmis:
    solvent_sentences.append(mol2alt_sentence(solvent_smi, 1))
    
print(solute_sentences[6])
print(solvent_sentences[6])

['2246728737', '3542456614', '2245384272', '2064788354', '2246699815', '4278941385', '864942730', '1510328189', '864662311', '1533864325']
['2246728737', '3537119515', '2245273601', '2406620359', '1016841875', '170898322', '1016841875', '170898322']


In [6]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('model_300dim.pkl')

solute_sequences = []
count = 0
sequences = []
solvent_sequences = []
Gsolv = []
l = len(solute_smis)

for i in range(l):
    flag = 0
    solute_substructures = []
    solvent_substructures = []
    
    for identifier in solute_sentences[i]:
        try:
            solute_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
    
    for identifier in solvent_sentences[i]:
        try:
            solvent_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
            
    if flag == 1:
        count += 1
        continue
    sequences.append((solute_substructures, solvent_substructures))
    solute_sequences.append(solute_substructures)
    solvent_sequences.append(solvent_substructures)
    Gsolv.append(energies[i])
    
    
print(count)
# solute_mol2vecs = [DfVec(x) for x in sentences2vec(solute_sentences, model, unseen='UNK')]
# solvent_mol2vecs = [DfVec(x) for x in sentences2vec(solvent_sentences, model, unseen='UNK')]

401


In [7]:
print(len(solute_sequences))
print(len(solvent_sequences))
print(len(Gsolv))
print(len(sequences))

# for i in solute_sequences:
#     print(np.asarray(i).shape)
# print('\n')
# for i in solvent_sequences:
#     print(np.asarray(i).shape)

2129
2129
2129
2129


In [8]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import math
import time
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer, LayerNorm, MultiheadAttention

In [ ]:
class TransformerModel(nn.Module):

    def __init__(self, ninp, nhead, nhid, nlayers, dropout=0):
        super(TransformerModel, self).__init__()
        
        self.model_type = 'Transformer'
        self.solv_src_mask = None
        self.solu_src_mask = None
        self.ninp = ninp
        
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        
        self.solv_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.solu_encoder = TransformerEncoder(encoder_layers, nlayers)
        
        self.solv_attn = MultiheadAttention(ninp, nhead, dropout=dropout)
        self.solu_attn = MultiheadAttention(ninp, nhead, dropout=dropout)
        
        self.fc1 = nn.Linear(600, 300)
        self.fc2 = nn.Linear(300, 1)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.fc1.bias.data.zero_()
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)

    def forward(self, solv_src, solu_src):
        if self.solv_src_mask is None or self.solv_src_mask.size(0) != len(solv_src):
            self.solv_src_mask = self._generate_square_subsequent_mask(len(solv_src))
            
        if self.solu_src_mask is None or self.solu_src_mask.size(0) != len(solu_src):
            self.solu_src_mask = self._generate_square_subsequent_mask(len(solu_src))
        
#         solv_src = self.pos_encoder(solv_src)
#         solu_src = self.pos_encoder(solu_src)

#         print(solv_src.shape)
#         print(solu_src.shape)
        
        solv_output = self.solv_encoder(solv_src, self.solv_src_mask)
        solu_output = self.solu_encoder(solu_src, self.solu_src_mask)
        
#         print(solv_output.shape)
#         print(solu_output.shape)

#         solv_multi = solv_output
#         solu_multi = solu_output
        
        solv_multi = self.solv_attn(solv_output, solu_output, solu_output)[0]
        solu_multi = self.solu_attn(solu_output, solv_output, solv_output)[0]
        
#         print(solv_multi.shape)
#         print(solu_multi.shape)
        
        solv_multi = torch.squeeze(solv_multi, 1)
        solu_multi = torch.squeeze(solu_multi, 1)
        
        inpu = torch.sum(solv_multi, dim=0)
        inpv = torch.sum(solu_multi, dim=0)
        
        inp = torch.cat((inpu, inpv), 0)
        
#         print(inp.shape)
        
        #mlp
        x = F.relu(self.fc1(inp))
        output = self.fc2(x)
        
#         print(output.shape)
#         print(output)
        
        return output

In [10]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [11]:
ninp = 300 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 3 # the number of heads in the multiheadattention models
dropout = 0 # the dropout value

In [12]:
model = TransformerModel(ninp, nhead, nhid, nlayers, dropout)

# PRINTING MODEL & PARAMETERS 
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (solv_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=200, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=200, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=200, bias=True)
 

In [13]:
learning_rate = 0.000002
nest_momentum = 0.009

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [19]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

num_epochs = 100
splits = 10

best_model = model
best_loss = float("inf")
losses = []

# Training and 10 fold cross validation
kf = KFold(n_splits = 10)
kf.get_n_splits(sequences)

for train_index, test_index in kf.split(sequences):
    model = TransformerModel(ninp, nhead, nhid, nlayers, dropout)

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
    
    seq_tr_va, seq_test = np.asarray(sequences)[train_index], np.asarray(sequences)[test_index]
    E_tr_va, E_test = np.asarray(Gsolv)[train_index], np.asarray(Gsolv)[test_index]
    seq_train, seq_val, E_train, E_val = train_test_split(seq_tr_va, E_tr_va, test_size=0.11)
    
    print("TRAIN:", len(seq_train), "VALIDATION:", len(seq_val), "TEST:", len(seq_test))
    
    #Training
    for epoch in range(num_epochs):   
        model.train()
        for i in range(len(seq_train)):
            solute, solvent = seq_train[i]
            energy = torch.FloatTensor([E_train[i]])
        
            # Forward pass
            solvent = torch.FloatTensor(solvent)
            solute = torch.FloatTensor(solute)
            
            solvent = torch.unsqueeze(solvent, 1)
            solute = torch.unsqueeze(solute, 1)
            
#             print(solvent.shape)
#             print(solute.shape)
#             print("into model")
            
            output = model(solvent, solute)
            print(output.shape)
            loss = criterion(output, energy)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            
        #Validation
        model.eval()
        outputs = []

        with torch.no_grad():
            for i in range(len(seq_val)):
                solute, solvent = seq_val[i]
                energy = torch.FloatTensor([E_val[i]])
                
                solvent = torch.FloatTensor(solvent)
                solute = torch.FloatTensor(solute)
            
                solvent = torch.unsqueeze(solvent, 1)
                solute = torch.unsqueeze(solute, 1)
        
                output = model(solvent, solute)
            
                outputs.append(output)
                
            val_loss = criterion(torch.squeeze(torch.FloatTensor(E_val), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
            print('Validation MSE: {}'.format(val_loss))
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = model
                
            scheduler.step()
            
    #Testing
    model.eval()
    outputs = []

    with torch.no_grad():
        for i in range(len(seq_test)):
            solute, solvent = seq_test[i]
            energy = torch.FloatTensor([E_test[i]])
            
            solvent = torch.FloatTensor(solvent)
            solute = torch.FloatTensor(solute)
            
            solvent = torch.unsqueeze(solvent, 1)
            solute = torch.unsqueeze(solute, 1)
        
            output = best_model(solvent, solute)
            outputs.append(output)
                
        test_loss = criterion(torch.squeeze(torch.FloatTensor(E_test), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
        print('Test MSE: {}'.format(test_loss))
        losses.append(test_loss)
    

TRAIN: 1705 VALIDATION: 211 TEST: 213
torch.Size([1])
Epoch [1/100], Loss: 2.7392
torch.Size([1])
Epoch [1/100], Loss: 20.3521
torch.Size([1])
Epoch [1/100], Loss: 12.2467
torch.Size([1])
Epoch [1/100], Loss: 4.8264
torch.Size([1])
Epoch [1/100], Loss: 0.0005
torch.Size([1])
Epoch [1/100], Loss: 35.1496
torch.Size([1])
Epoch [1/100], Loss: 25.7190
torch.Size([1])
Epoch [1/100], Loss: 223.6584
torch.Size([1])
Epoch [1/100], Loss: 50.1960
torch.Size([1])
Epoch [1/100], Loss: 33.3512
torch.Size([1])
Epoch [1/100], Loss: 102.2087
torch.Size([1])
Epoch [1/100], Loss: 181.3101
torch.Size([1])
Epoch [1/100], Loss: 94.1101
torch.Size([1])
Epoch [1/100], Loss: 56.0861
torch.Size([1])
Epoch [1/100], Loss: 19.0167
torch.Size([1])
Epoch [1/100], Loss: 21.0921
torch.Size([1])
Epoch [1/100], Loss: 0.5022
torch.Size([1])
Epoch [1/100], Loss: 5.4754
torch.Size([1])
Epoch [1/100], Loss: 16.7352
torch.Size([1])
Epoch [1/100], Loss: 113.0905
torch.Size([1])
Epoch [1/100], Loss: 0.0000
torch.Size([1])
Epo

Epoch [1/100], Loss: 70.2414
torch.Size([1])
Epoch [1/100], Loss: 3.6542
torch.Size([1])
Epoch [1/100], Loss: 2.0923
torch.Size([1])
Epoch [1/100], Loss: 0.3997
torch.Size([1])
Epoch [1/100], Loss: 12.8492
torch.Size([1])
Epoch [1/100], Loss: 19.2166
torch.Size([1])
Epoch [1/100], Loss: 37.8386
torch.Size([1])
Epoch [1/100], Loss: 23.0662
torch.Size([1])
Epoch [1/100], Loss: 1.1037
torch.Size([1])
Epoch [1/100], Loss: 45.4042
torch.Size([1])
Epoch [1/100], Loss: 6.9877
torch.Size([1])
Epoch [1/100], Loss: 14.9685
torch.Size([1])
Epoch [1/100], Loss: 23.5118
torch.Size([1])
Epoch [1/100], Loss: 0.3766
torch.Size([1])
Epoch [1/100], Loss: 0.1598
torch.Size([1])
Epoch [1/100], Loss: 28.9988
torch.Size([1])
Epoch [1/100], Loss: 20.8620
torch.Size([1])
Epoch [1/100], Loss: 17.4989
torch.Size([1])
Epoch [1/100], Loss: 5.0510
torch.Size([1])
Epoch [1/100], Loss: 1.0445
torch.Size([1])
Epoch [1/100], Loss: 10.9152
torch.Size([1])
Epoch [1/100], Loss: 10.4713
torch.Size([1])
Epoch [1/100], Loss

torch.Size([1])
Epoch [1/100], Loss: 7.5819
torch.Size([1])
Epoch [1/100], Loss: 42.2908
torch.Size([1])
Epoch [1/100], Loss: 23.7334
torch.Size([1])
Epoch [1/100], Loss: 37.4786
torch.Size([1])
Epoch [1/100], Loss: 2.6617
torch.Size([1])
Epoch [1/100], Loss: 7.3420
torch.Size([1])
Epoch [1/100], Loss: 0.3328
torch.Size([1])
Epoch [1/100], Loss: 2.9121
torch.Size([1])
Epoch [1/100], Loss: 51.1558
torch.Size([1])
Epoch [1/100], Loss: 4.8206
torch.Size([1])
Epoch [1/100], Loss: 12.9626
torch.Size([1])
Epoch [1/100], Loss: 1.5740
torch.Size([1])
Epoch [1/100], Loss: 0.0066
torch.Size([1])
Epoch [1/100], Loss: 13.1117
torch.Size([1])
Epoch [1/100], Loss: 5.1605
torch.Size([1])
Epoch [1/100], Loss: 8.8636
torch.Size([1])
Epoch [1/100], Loss: 13.5851
torch.Size([1])
Epoch [1/100], Loss: 28.2043
torch.Size([1])
Epoch [1/100], Loss: 12.7228
torch.Size([1])
Epoch [1/100], Loss: 2.4040
torch.Size([1])
Epoch [1/100], Loss: 17.0420
torch.Size([1])
Epoch [1/100], Loss: 1.0787
torch.Size([1])
Epoch 

Epoch [1/100], Loss: 12.7683
torch.Size([1])
Epoch [1/100], Loss: 17.7754
torch.Size([1])
Epoch [1/100], Loss: 1.7215
torch.Size([1])
Epoch [1/100], Loss: 10.0952
torch.Size([1])
Epoch [1/100], Loss: 2.1520
torch.Size([1])
Epoch [1/100], Loss: 0.6736
torch.Size([1])
Epoch [1/100], Loss: 98.7605
torch.Size([1])
Epoch [1/100], Loss: 0.4717
torch.Size([1])
Epoch [1/100], Loss: 5.3621
torch.Size([1])
Epoch [1/100], Loss: 17.1920
torch.Size([1])
Epoch [1/100], Loss: 19.0682
torch.Size([1])
Epoch [1/100], Loss: 56.3392
torch.Size([1])
Epoch [1/100], Loss: 0.0450
torch.Size([1])
Epoch [1/100], Loss: 20.5236
torch.Size([1])
Epoch [1/100], Loss: 0.1215
torch.Size([1])
Epoch [1/100], Loss: 3.0198
torch.Size([1])
Epoch [1/100], Loss: 43.6775
torch.Size([1])
Epoch [1/100], Loss: 6.4875
torch.Size([1])
Epoch [1/100], Loss: 15.3727
torch.Size([1])
Epoch [1/100], Loss: 13.5291
torch.Size([1])
Epoch [1/100], Loss: 4.7561
torch.Size([1])
Epoch [1/100], Loss: 0.3947
torch.Size([1])
Epoch [1/100], Loss: 

Epoch [1/100], Loss: 2.9503
torch.Size([1])
Epoch [1/100], Loss: 9.2061
torch.Size([1])
Epoch [1/100], Loss: 2.3452
torch.Size([1])
Epoch [1/100], Loss: 0.2906
torch.Size([1])
Epoch [1/100], Loss: 2.5461
torch.Size([1])
Epoch [1/100], Loss: 0.4643
torch.Size([1])
Epoch [1/100], Loss: 0.3673
torch.Size([1])
Epoch [1/100], Loss: 1.6101
torch.Size([1])
Epoch [1/100], Loss: 3.3052
torch.Size([1])
Epoch [1/100], Loss: 0.2435
torch.Size([1])
Epoch [1/100], Loss: 6.0251
torch.Size([1])
Epoch [1/100], Loss: 2.5584
torch.Size([1])
Epoch [1/100], Loss: 0.0008
torch.Size([1])
Epoch [1/100], Loss: 0.3867
torch.Size([1])
Epoch [1/100], Loss: 2.1492
torch.Size([1])
Epoch [1/100], Loss: 15.4739
torch.Size([1])
Epoch [1/100], Loss: 0.0977
torch.Size([1])
Epoch [1/100], Loss: 15.8975
torch.Size([1])
Epoch [1/100], Loss: 0.9982
torch.Size([1])
Epoch [1/100], Loss: 0.1503
torch.Size([1])
Epoch [1/100], Loss: 10.7201
torch.Size([1])
Epoch [1/100], Loss: 0.1763
torch.Size([1])
Epoch [1/100], Loss: 0.1391
t

...

torch.Size([1])
Epoch [22/100], Loss: 0.0691
torch.Size([1])
Epoch [22/100], Loss: 0.4523
torch.Size([1])
Epoch [22/100], Loss: 0.1848
torch.Size([1])
Epoch [22/100], Loss: 5.0399
torch.Size([1])
Epoch [22/100], Loss: 0.6888
torch.Size([1])
Epoch [22/100], Loss: 0.0013
torch.Size([1])
Epoch [22/100], Loss: 11.1696
torch.Size([1])
Epoch [22/100], Loss: 1.2610
torch.Size([1])
Epoch [22/100], Loss: 0.6069
torch.Size([1])
Epoch [22/100], Loss: 1.3148
torch.Size([1])
Epoch [22/100], Loss: 0.0150
torch.Size([1])
Epoch [22/100], Loss: 0.2706
torch.Size([1])
Epoch [22/100], Loss: 0.6327
torch.Size([1])
Epoch [22/100], Loss: 3.9703
torch.Size([1])
Epoch [22/100], Loss: 0.1509
torch.Size([1])
Epoch [22/100], Loss: 0.1805
torch.Size([1])
Epoch [22/100], Loss: 1.1266
torch.Size([1])
Epoch [22/100], Loss: 3.9755
torch.Size([1])
Epoch [22/100], Loss: 0.3303
torch.Size([1])
Epoch [22/100], Loss: 0.1050
torch.Size([1])
Epoch [22/100], Loss: 0.0430
torch.Size([1])
Epoch [22/100], Loss: 1.8391
torch.Siz

Epoch [22/100], Loss: 0.0092
torch.Size([1])
Epoch [22/100], Loss: 0.3198
torch.Size([1])
Epoch [22/100], Loss: 0.1193
torch.Size([1])
Epoch [22/100], Loss: 44.1704
torch.Size([1])
Epoch [22/100], Loss: 0.3325
torch.Size([1])
Epoch [22/100], Loss: 0.3215
torch.Size([1])
Epoch [22/100], Loss: 0.7035
torch.Size([1])
Epoch [22/100], Loss: 0.0509
torch.Size([1])
Epoch [22/100], Loss: 0.3900
torch.Size([1])
Epoch [22/100], Loss: 1.8584
torch.Size([1])
Epoch [22/100], Loss: 0.6682
torch.Size([1])
Epoch [22/100], Loss: 0.8726
torch.Size([1])
Epoch [22/100], Loss: 10.6924
torch.Size([1])
Epoch [22/100], Loss: 0.2254
torch.Size([1])
Epoch [22/100], Loss: 0.8765
torch.Size([1])
Epoch [22/100], Loss: 0.6458
torch.Size([1])
Epoch [22/100], Loss: 1.3379
torch.Size([1])
Epoch [22/100], Loss: 5.5530
torch.Size([1])
Epoch [22/100], Loss: 1.9708
torch.Size([1])
Epoch [22/100], Loss: 3.7248
torch.Size([1])
Epoch [22/100], Loss: 0.3382
torch.Size([1])
Epoch [22/100], Loss: 0.0407
torch.Size([1])
Epoch [2

torch.Size([1])
Epoch [22/100], Loss: 1.2821
torch.Size([1])
Epoch [22/100], Loss: 2.6446
torch.Size([1])
Epoch [22/100], Loss: 0.1277
torch.Size([1])
Epoch [22/100], Loss: 0.0647
torch.Size([1])
Epoch [22/100], Loss: 0.1834
torch.Size([1])
Epoch [22/100], Loss: 0.8926
torch.Size([1])
Epoch [22/100], Loss: 0.4690
torch.Size([1])
Epoch [22/100], Loss: 0.6527
torch.Size([1])
Epoch [22/100], Loss: 0.0023
torch.Size([1])
Epoch [22/100], Loss: 0.1066
torch.Size([1])
Epoch [22/100], Loss: 1.1432
torch.Size([1])
Epoch [22/100], Loss: 1.8230
torch.Size([1])
Epoch [22/100], Loss: 0.5934
torch.Size([1])
Epoch [22/100], Loss: 0.0030
torch.Size([1])
Epoch [22/100], Loss: 0.2736
torch.Size([1])
Epoch [22/100], Loss: 0.2770
torch.Size([1])
Epoch [22/100], Loss: 2.9453
torch.Size([1])
Epoch [22/100], Loss: 0.0103
torch.Size([1])
Epoch [22/100], Loss: 1.8052
torch.Size([1])
Epoch [22/100], Loss: 0.0000
torch.Size([1])
Epoch [22/100], Loss: 1.1906
torch.Size([1])
Epoch [22/100], Loss: 0.0743
torch.Size

Validation MSE: 2.3362393379211426
torch.Size([1])
Epoch [23/100], Loss: 0.3885
torch.Size([1])
Epoch [23/100], Loss: 0.1949
torch.Size([1])
Epoch [23/100], Loss: 0.3060
torch.Size([1])
Epoch [23/100], Loss: 0.6995
torch.Size([1])
Epoch [23/100], Loss: 42.3825
torch.Size([1])
Epoch [23/100], Loss: 2.0424
torch.Size([1])
Epoch [23/100], Loss: 1.1323
torch.Size([1])
Epoch [23/100], Loss: 0.0116
torch.Size([1])
Epoch [23/100], Loss: 5.0920
torch.Size([1])
Epoch [23/100], Loss: 0.0183
torch.Size([1])
Epoch [23/100], Loss: 0.2621
torch.Size([1])
Epoch [23/100], Loss: 0.0000
torch.Size([1])
Epoch [23/100], Loss: 2.2875
torch.Size([1])
Epoch [23/100], Loss: 0.1951
torch.Size([1])
Epoch [23/100], Loss: 10.0645
torch.Size([1])
Epoch [23/100], Loss: 0.9519
torch.Size([1])
Epoch [23/100], Loss: 0.5028
torch.Size([1])
Epoch [23/100], Loss: 0.5855
torch.Size([1])
Epoch [23/100], Loss: 1.8845
torch.Size([1])
Epoch [23/100], Loss: 0.2567
torch.Size([1])
Epoch [23/100], Loss: 0.3217
torch.Size([1])
Ep

KeyboardInterrupt: 

In [61]:
print(losses)

with open('losses.txt', 'a') as f:
    for item in losses:
        f.write("%s\n" % item)

[tensor([-3.1967]), tensor([-3.5724]), tensor([-3.4067]), tensor([-3.3865]), tensor([-3.7676])]
[-5.28, -6.88, -5.86, -6.17, -6.64]


In [ ]:
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')